In [315]:
# import the libraries
import pandas as pd
import numpy as np
#Text Processing Imports
import string
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.linear_model import LinearRegression
from sklearn import metrics

import nltk
nltk.download('stopwords')
  

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hazimsaeed/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [316]:
df = pd.read_excel("mealkit_items_2.xlsx")

In [317]:
# Descripvtive Statistics

In [318]:
df.head()

,merchant_name,item_key,product_title,product_subtitle,product_price
0,PEACHDISH,227272687,ZEN Modern Asian Bistro 2 - Orange Chicken Combo,"Chicken, scallions, onions, orange peels, chil...",10.99
1,PEACHDISH,227273654,Pho O Sha - Tofu Vegetable Pho,Tofu and assorted seasonal vegetables may inc...,8.99
2,PEACHDISH,227372050,TIG Restaurant - Chicken Katsu Combo,"Fried chicken cutlet served with white rice, s...",8.99
3,PEACHDISH,227421173,Pick Up Stix Mira Mesa - 2 Chicken Eggrolls,2 delicious crispy egg rolls filled with chick...,3.99
4,PEACHDISH,230993729,Big Thyme Sandwich Company - Baby Kale Salad W...,"Chicken, avocado, cucumber, walnuts, and straw...",10.99


In [319]:
df.describe()

,item_key,product_price
count,1.627000e+03,1079.000000
mean,7.166577e+08,12.961418
std,8.048195e+08,9.589569
min,2.335910e+06,0.500000
25%,9.727199e+07,9.495000
50%,1.350837e+08,10.490000
75%,1.783910e+09,11.490000
max,2.055253e+09,86.000000


In [320]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1627 entries, 0 to 1626
Data columns (total 5 columns):
merchant_name       1627 non-null object
item_key            1627 non-null int64
product_title       1627 non-null object
product_subtitle    880 non-null object
product_price       1079 non-null float64
dtypes: float64(1), int64(1), object(3)
memory usage: 63.6+ KB


In [321]:
df.isnull().sum()

merchant_name         0
item_key              0
product_title         0
product_subtitle    747
product_price       548
dtype: int64

In [322]:
df.isnull().sum()

merchant_name         0
item_key              0
product_title         0
product_subtitle    747
product_price       548
dtype: int64

In [323]:
df.shape

(1627, 5)

In [324]:
df.product_price.count()

1079

In [325]:
# function for text processing
def process_text(text):
    no_punctuation = [character for character in text if character not in string.punctuation]
    no_punctuation = ''.join(no_punctuation)
    return [word for word in no_punctuation.split() if word.lower() not in stopwords.words('english')]


we can build three model one using product_title and one using	product_subtitle and one using both product_title and product_subtitle

In [326]:
# split train and test
train = df[df.product_price.isnull()==True]
test = df[df.product_price.isnull()==False]

In [327]:
train.shape

(548, 5)

In [328]:
test.shape

(1079, 5)

In [329]:
# model using product title
bow_transformer = CountVectorizer(analyzer=process_text).fit(df.product_title)
descriptions_bow_product_title = bow_transformer.transform(df.product_title)


In [330]:
descriptions_bow_product_title.shape


(1627, 1732)

In [331]:
tfidf_transformer = TfidfTransformer().fit(descriptions_bow_product_title)


In [332]:
product_title_tfidf = tfidf_transformer.transform(descriptions_bow_product_title)


In [333]:
print(product_title_tfidf[0])


  (0, 1708)	0.4152483653703807
  (0, 1026)	0.4081893773064363
  (0, 951)	0.3856462691284971
  (0, 372)	0.28814432411196883
  (0, 315)	0.14882502664327005
  (0, 159)	0.34417162033838783
  (0, 77)	0.39061138616128743
  (0, 4)	0.3688085995474811


In [334]:
train_indices = df.loc[df.product_price.isnull()==False].index


In [335]:
product_title_tfidf

<1627x1732 sparse matrix of type '<class 'numpy.float64'>'
	with 10252 stored elements in Compressed Sparse Row format>

In [336]:
y = df.loc[train_indices, 'product_price']

In [337]:
X = product_title_tfidf[list(train_indices)]


In [338]:
lm = LinearRegression()
lm.fit(X,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [339]:
predictions = lm.predict(X)


In [340]:
np.sqrt(metrics.mean_squared_error(y, predictions))


0.39560695581157557

In [349]:
# predict unknow prices
pred_indices = df.loc[df.product_price.isnull()==True].index

X_predict = product_title_tfidf[list(pred_indices)]
predictions_title = lm.predict(X_predict)
predictions_title

array([ 27.50229902,  48.21184051,   0.73324537,  45.41974254,
        25.02692368,  20.40705051,  12.36594507,  43.70959485,
        40.77957074,  10.09124631,  24.29306051,  18.11186702,
        23.72316038,  12.71126794,  20.32039028,   6.92620286,
        29.48304953,  24.24213242,  25.14784602,  24.83302419,
        17.69146082,  40.74013072,   4.54753731,  21.69340881,
        33.56981618,  21.39078633,  51.2757974 ,   4.79575832,
        24.74717808,  15.21263164,  16.81758852,  12.0229866 ,
         0.60615964,  21.81772537,  26.42707882,  30.08106111,
        12.61492871,  17.89588747,   9.55396498,  28.83720379,
        16.69747567,   6.17317791,  26.29768219,  27.98639823,
        12.77548507,   2.88664771,  22.66848641,  28.80556813,
        13.67768213,  18.28448025,  17.59185776,  29.80265073,
         5.59314798,  13.75138986,  13.92892778,   7.51322916,
        22.87344616,  25.00000048,  46.16099291,  19.46900046,
        13.11306818,  18.25484331,  17.00466134,  23.97

# Using only product title we get 39.56% accuracy

In [350]:
# remove where subtitle is missing
df = df[df.product_subtitle.isnull()==False]
df.shape
df.reset_index(inplace=True)

In [351]:
# index
train_indices = df.loc[df.product_price.isnull()==False].index


In [352]:
# model using product sub title
bow_transformer = CountVectorizer(analyzer=process_text).fit(df.product_subtitle)
descriptions_bow_product_sub_title = bow_transformer.transform(df.product_subtitle)


In [353]:
descriptions_bow_product_sub_title.shape


(880, 1745)

In [354]:
tfidf_transformer_sub_title = TfidfTransformer().fit(descriptions_bow_product_sub_title)


In [355]:
product_sub_title_tfidf = tfidf_transformer_sub_title.transform(descriptions_bow_product_sub_title)


In [356]:
product_sub_title_tfidf.shape

(880, 1745)

In [357]:
print(product_sub_title_tfidf[0])


  (0, 1742)	0.255209618628
  (0, 1740)	0.29911244475680576
  (0, 1720)	0.2862355286488866
  (0, 1598)	0.14126928040956868
  (0, 1566)	0.18316968648105605
  (0, 1457)	0.19315779858622678
  (0, 1448)	0.29911244475680576
  (0, 1447)	0.08299264831597396
  (0, 1404)	0.08698512506530279
  (0, 1356)	0.1562424088817743
  (0, 1307)	0.16075452061266832
  (0, 1299)	0.29911244475680576
  (0, 1248)	0.48191066913481784
  (0, 1247)	0.1267762708416356
  (0, 1025)	0.2241837086071134
  (0, 891)	0.1891466753352635
  (0, 753)	0.1799325466617642
  (0, 100)	0.15443971184153735
  (0, 6)	0.19604660258897524


In [358]:
train_indices = df.loc[df.product_price.isnull()==False].index


In [359]:
product_sub_title_tfidf

<880x1745 sparse matrix of type '<class 'numpy.float64'>'
	with 14347 stored elements in Compressed Sparse Row format>

In [360]:
y = df.loc[train_indices, 'product_price']

In [361]:
X = product_sub_title_tfidf[list(train_indices)]


In [362]:
lm = LinearRegression()
lm.fit(X,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [363]:
predictions = lm.predict(X)


In [364]:
np.sqrt(metrics.mean_squared_error(y, predictions))


0.22515093366032585

In [367]:
predictions

array([10.4900001 ,  8.9900001 ,  8.98999992,  3.98999991, 10.99000028,
       11.48999969,  7.00000016, 10.98999993,  8.74500015,  7.98999992,
       10.74000017,  7.99000014,  3.99000024,  9.98999988,  7.98999993,
        7.99000014,  7.99000019,  7.98999997,  7.98999996,  7.99000012,
        9.99000016,  9.49000013,  3.50000039, 10.99000003, 10.48999949,
        9.98999989,  2.9899999 ,  9.99000041, 10.73999997,  9.9900001 ,
       10.98999978,  7.98999977,  9.99000006,  9.98999984,  8.99000016,
        9.99000001,  7.99000007,  3.5       ,  7.98999997, 11.74999993,
       10.73999997,  9.99000004,  9.77050136,  9.49000004, 11.24000012,
        9.98999899,  7.99000019, 10.75000022,  9.98999996,  9.98999976,
        9.99000024, 10.98999985,  3.99999992, 10.9899996 ,  9.98999992,
       10.48999997,  9.98999987,  9.99000006,  9.98999987, 11.74999984,
       11.48999976, 12.25000005,  9.99000012,  3.98999991,  3.99000004,
       10.99000006, 11.99000041,  9.24000029, 10.98999955, 10.99

# Model using only sub title has 22.51% Prediction accuracy

In [295]:
# join two columns
df['title_sub_title'] = df.product_title + df.product_subtitle

In [296]:
# model using product title and subtitle
bow_transformer = CountVectorizer(analyzer=process_text).fit(df.title_sub_title)
descriptions_bow_product_title_sub_title = bow_transformer.transform(df.title_sub_title)


In [297]:
descriptions_bow_product_title_sub_title.shape


(880, 2827)

In [298]:
tfidf_transformer = TfidfTransformer().fit(descriptions_bow_product_title_sub_title)


In [299]:
product_title_sub_title_tfidf = tfidf_transformer.transform(descriptions_bow_product_title_sub_title)


In [300]:
print(product_title_sub_title_tfidf[0])


  (0, 2824)	0.21354222135895518
  (0, 2822)	0.2502771495559467
  (0, 2802)	0.239502613373805
  (0, 2680)	0.11820461983610635
  (0, 2647)	0.1532640577853859
  (0, 2538)	0.1616214373292556
  (0, 2529)	0.2502771495559467
  (0, 2528)	0.0694426588352389
  (0, 2483)	0.07278329449920411
  (0, 2435)	0.13073312535183462
  (0, 2385)	0.13450855657270633
  (0, 2377)	0.2502771495559467
  (0, 2326)	0.403230392870228
  (0, 2325)	0.1060778454850809
  (0, 2101)	0.1875818293441054
  (0, 1967)	0.1582651995285018
  (0, 1828)	0.15055543719507536
  (0, 1625)	0.20913090170838883
  (0, 914)	0.20913090170838883
  (0, 851)	0.19258297108722128
  (0, 398)	0.239502613373805
  (0, 313)	0.07657196347891977
  (0, 139)	0.17398995572080378
  (0, 57)	0.1773971152545132
  (0, 5)	0.3065281155707718


In [301]:
train_indices = df.loc[df.product_price.isnull()==False].index


In [302]:
product_title_sub_title_tfidf

<880x2827 sparse matrix of type '<class 'numpy.float64'>'
	with 18691 stored elements in Compressed Sparse Row format>

In [303]:
y = df.loc[train_indices, 'product_price']

In [304]:
X = product_title_sub_title_tfidf[list(train_indices)]

In [311]:
lm = LinearRegression()
lm.fit(X,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [312]:
predictions = lm.predict(X)


In [307]:
np.sqrt(metrics.mean_squared_error(y, predictions))


0.13034882585633567

In [368]:
predictions

array([10.4900001 ,  8.9900001 ,  8.98999992,  3.98999991, 10.99000028,
       11.48999969,  7.00000016, 10.98999993,  8.74500015,  7.98999992,
       10.74000017,  7.99000014,  3.99000024,  9.98999988,  7.98999993,
        7.99000014,  7.99000019,  7.98999997,  7.98999996,  7.99000012,
        9.99000016,  9.49000013,  3.50000039, 10.99000003, 10.48999949,
        9.98999989,  2.9899999 ,  9.99000041, 10.73999997,  9.9900001 ,
       10.98999978,  7.98999977,  9.99000006,  9.98999984,  8.99000016,
        9.99000001,  7.99000007,  3.5       ,  7.98999997, 11.74999993,
       10.73999997,  9.99000004,  9.77050136,  9.49000004, 11.24000012,
        9.98999899,  7.99000019, 10.75000022,  9.98999996,  9.98999976,
        9.99000024, 10.98999985,  3.99999992, 10.9899996 ,  9.98999992,
       10.48999997,  9.98999987,  9.99000006,  9.98999987, 11.74999984,
       11.48999976, 12.25000005,  9.99000012,  3.98999991,  3.99000004,
       10.99000006, 11.99000041,  9.24000029, 10.98999955, 10.99

# Product price using product title and subtitle has 13.03% prediction accuracy.

## Based on the 3 models product price is best predicted by product title